# Setup Enviroment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My Drive/Thesis/

In [ ]:
! pip install import-ipynb

In [ ]:
import import_ipynb as ipy

In [ ]:
import Thesis_Undirected_GPARs_Final as undirected
import Thesis_Directed_GPARs_Final as directed
import Thesis_Measure as measure

In [ ]:
import time
import re
import math
import queue
import random
import json
import multiprocessing
import os
import networkx
from collections import defaultdict
import copy
import progressbar

# Split K Fold

In [ ]:
# convert from drawDataFormat to GramiFormat
def convert_Grami(src_nodes, src_edges, start_idx, split_char):
    des_path = src_nodes.split('.')[0] + '.lg'
    with open(src_nodes, 'r') as node_f:
        with open(src_edges, 'r') as edge_f:
            with open(des_path, 'w') as des_f:
                des_f.write('t\t#\t1\n')
                
                nodes_set = set()
                edges_set = set()
                nodes = node_f.readlines()
                for node in nodes:
                    data = node.strip('\n').split(split_char)
                    nodes_set.add((int(data[0])-start_idx, data[1]))
                edges = edge_f.readlines()
                for edge in edges:
                    data = edge.strip('\n').split(split_char)
                    if data[0] != data[1]:
                        edges_set.add((int(data[0])-start_idx, int(data[1])-start_idx))
                
                nodes_set = sorted(nodes_set)
                print('Lennodes: ', len(nodes_set))
                edges_set = sorted(edges_set)
                print('Lenedges: ', len(edges_set))
                for node in nodes_set:
                    v, l = node
                    des_f.write('v {} {}\n'.format(v, l))
                for edge in edges_set:
                    v1, v2 = edge
                    des_f.write('e {} {} -1\n'.format(v1, v2))

# convert_Grami('kFoldData/CL_10M_1d8_L5/CL_10M_1d8_L5.node_labels', 'kFoldData/CL_10M_1d8_L5/CL_10M_1d8_L5.edges', 1, '\t')

In [ ]:
def device_data(profiles_filename, relationships_filename, num_partition):
  def device_edges():
    print('Split test data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_partition):
      map_idx[i] = set()
      edges_old.append([])

    # des_filename = relationships_filename.split('.')[0]
    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_partition ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_partition)
        
      lines = set(lines)

      for i in range(0, num_partition-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        # des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          # des_f.write('{}\t{}\n'.format(data[0], data[1]))
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])
        # des_f.close()

      # des_f = open(des_filename+'_'+str(num_partition-1)+'.txt', 'w')
      print('Partition ', num_partition-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        # des_f.write('{}\t{}\n'.format(data[0], data[1]))
        edges_old[num_partition-1].append([data[0], data[1]])
        map_idx[num_partition-1].add(data[0])
        map_idx[num_partition-1].add(data[1])
      # des_f.close()

    print("Done!")
    return map_idx, edges_old

  def modified_vertex_idx(map_idx):
    map_new = {} #{node_index: {file_idx: node_index_modified},....}
    for key in map_idx:
      idx = 0
      for vertex in map_idx[key]:
        if vertex not in map_new:
          map_new[vertex] = {key: idx}
        else:
          map_new[vertex][key] = idx
        idx += 1
    return map_new

  def device_vertex():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]
    
      print('Split edge ........')
      map_idx, edges_old = device_edges() #{file_index: set(vertex_idxs)}
      print('Modify vertex index ............')
      new_map = modified_vertex_idx(map_idx) #{node_index_old: {file_idx: node_index_modified},....}
      # print(map_idx)
      # print(edges_old)
      # print(new_map)

      # Split and update index profile
      print('Update index profiles:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_partition):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Update index edges in relations_file_splited
      print('Update index relationships:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_partition):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex()

In [ ]:
def device_k_fold(profiles_filename, relationships_filename, num_fold):
  def device_edges_():
    print('Split train data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_fold):
      map_idx[i] = set()
      edges_old.append([])

    # des_filename = relationships_filename.split('.')[0]
    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_fold ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_fold)
    
      lines = set(lines)

      for i in range(0, num_fold-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        # des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          # des_f.write('{}\t{}\n'.format(data[0], data[1]))
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])
        # des_f.close()

      # des_f = open(des_filename+'_'+str(num_partition-1)+'.txt', 'w')
      print('Partition ', num_fold-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        # des_f.write('{}\t{}\n'.format(data[0], data[1]))
        edges_old[num_fold-1].append([data[0], data[1]])
        map_idx[num_fold-1].add(data[0])
        map_idx[num_fold-1].add(data[1])
      # des_f.close()

    print("Done!")
    return map_idx, edges_old

  def split_k_fold(map_idx, edges_old):
    # map_idx: {file_index: set(vertex_idxs)}
    # edges_old #edges[file_index] = [[v0,v1], ....]

    fold_map = {} #{node_index: {fold_idx: node_index_modified},....}
    map_idx_new = {} # map_idx_new: {fold_index: set(vertex_idxs)}
    edges_old_new = []

    for i in range(0, num_fold):
      map_idx_new[i] = set()
      edges_old_new.append([])

    # map_idx_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          map_idx_new[i] = set(map_idx_new[i] | map_idx[j])
    # edges_old_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          edges_old_new[i] = edges_old_new[i] + edges_old[j]

    # map_idx_new: {fold_index: set(vertex_idxs)}
    for fold in map_idx_new:
      idx = 0
      for vertex in map_idx_new[fold]:
        if vertex not in fold_map:
          fold_map[vertex] = {fold: idx}
        else:
          fold_map[vertex][fold] = idx
        idx += 1

    # print(fold_map)
    # print(map_idx_new)
    # print(edges_old_new)
    return fold_map, map_idx_new, edges_old_new

  def device_vertex_():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]

      print('Split edge ..........')
      map_idx, edges_old = device_edges_() #{file_index: set(vertex_idxs)}

      edge = edges_old
      print('Modify vertex index ............')
      new_map, map_idx, edges_old = split_k_fold(map_idx, edges_old) #{node_index_old: {file_idx: node_index_modified},....}

      # Split and update index profile
      print('Update index profiles:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Update index of edges in relationships_file_splited
      print('Update index relationships:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex_()

In [ ]:
# both train and test
def device_k_fold_v1(profiles_filename, relationships_filename, num_fold):
  def device_edges_():
    print('Split train data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_fold):
      map_idx[i] = set()
      edges_old.append([])

    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_fold ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_fold)
    
      lines = set(lines)

      for i in range(0, num_fold-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])

      print('Partition ', num_fold-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        edges_old[num_fold-1].append([data[0], data[1]])
        map_idx[num_fold-1].add(data[0])
        map_idx[num_fold-1].add(data[1])

    print("Done!")
    return map_idx, edges_old

  # Test
  def modified_vertex_idx(map_idx):
    map_new = {} #{node_index: {file_idx: node_index_modified},....}
    for key in map_idx:
      idx = 0
      for vertex in map_idx[key]:
        if vertex not in map_new:
          map_new[vertex] = {key: idx}
        else:
          map_new[vertex][key] = idx
        idx += 1
    return map_new

  # Train
  def split_k_fold(map_idx, edges_old):
    # map_idx: {file_index: set(vertex_idxs)}
    # edges_old #edges[file_index] = [[v0,v1], ....]

    fold_map = {} #{node_index: {fold_idx: node_index_modified},....}
    map_idx_new = {} # map_idx_new: {fold_index: set(vertex_idxs)}
    edges_old_new = []

    for i in range(0, num_fold):
      map_idx_new[i] = set()
      edges_old_new.append([])

    # map_idx_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          map_idx_new[i] = set(map_idx_new[i] | map_idx[j])
    # edges_old_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          edges_old_new[i] = edges_old_new[i] + edges_old[j]

    # map_idx_new: {fold_index: set(vertex_idxs)}
    for fold in map_idx_new:
      idx = 0
      for vertex in map_idx_new[fold]:
        if vertex not in fold_map:
          fold_map[vertex] = {fold: idx}
        else:
          fold_map[vertex][fold] = idx
        idx += 1

    return fold_map, map_idx_new, edges_old_new

  def device_vertex_():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]

      print('Split edge ..........')
      map_idx_test, edges_old_test = device_edges_() #{file_index: set(vertex_idxs)}
      # Test
      print('Test - Modify vertex index ............')
      new_map_test = modified_vertex_idx(map_idx_test) #{node_index_old: {file_idx: node_index_modified},....}
      # train
      # edge = edges_old
      print('Train - Modify vertex index ............')
      new_map, map_idx, edges_old = split_k_fold(map_idx_test, edges_old_test) #{node_index_old: {file_idx: node_index_modified},....}

      # Train
      # Split and update index profile
      print('Update index profiles - train:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Test
      # Split and update index profile
      print('Update index profiles - test:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_test.txt', 'w')
        vertex_set = set()
        for vertex in map_idx_test[i]:
          vertex_set.add((new_map_test[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Train
      # Update index of edges in relationships_file_splited
      print('Update index relationships - train:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()



      # Test
      # Update index edges in relations_file_splited
      print('Update index relationships - test:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_test.txt', 'w')
        edge_set = set()
        for edge in edges_old_test[i]:
          # print(edge)
          edge_set.add((new_map_test[edge[0]][i], new_map_test[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex_()

In [ ]:
# print('Test: ')
# device_data('kFoldData/test2/test2_profiles.txt', 'kFoldData/test2/test2_relationships.txt', 5)
# print('Train: ')
# device_k_fold('kFoldData/test2/test2_profiles.txt', 'kFoldData/test2/test2_relationships.txt', 5)
# Split both train and test
# device_k_fold_v1('kFoldData/cora/cora_profiles.txt', 'kFoldData/cora/cora_relationships.txt', 4)

In [ ]:
# Convert Data GramiFormat -> MyFormat / MyFormat -> GramiFormat

In [ ]:
Grami_source_path = 'kFoldData/facebook/'
Grami_des_path = 'kFoldData/facebook/'

# convert data from gramiFormat to myDataFormat
def data_convert_from_Grami(source_filename, des_path): #des_profile_filename, des_relationships_filename):
  des_profile_filename = source_filename.split('.')[0] + '_profiles.txt' 
  des_relationships_filename = source_filename.split('.')[0] + '_relationships.txt'
  with open(Grami_source_path + source_filename, 'r') as f:
    with open(Grami_des_path + des_profile_filename, 'w') as profiles_f:
      with open(Grami_des_path + des_relationships_filename, 'w') as relationships_f:
        vertexs_set = set()
        edges_set = set()
        vertexs_idx = set()

        lines = f.readlines()
        for line in lines:
          data = line.strip('\n').split(' ')
          if data[0] == 'v':
            vertexs_idx.add(data[1])
            vertexs_set.add((int(data[1]), int(data[2])))
            # profiles_f.write('{}\t{}'.format(data[1], data[2]))
          if data[0] == 'e':
            if data[1] in vertexs_idx and data[2] in vertexs_idx:
              edges_set.add((int(data[1]), int(data[2])))
            # relationships_f.write('{}\t{}\n'.format(data[1], data[2]))
        vertexs_set = sorted(vertexs_set)
        edges_set = sorted(edges_set)
        for vertex in vertexs_set:
          v, l = vertex
          profiles_f.write('{}\t{}\n'.format(v, l))
        print('Vertex OK!')
        for edge in edges_set:
          v1, v2 = edge
          relationships_f.write('{}\t{}\n'.format(v1, v2))
        print('Edge OK!')
    print('Converted from {} to {} and {}!'.format(Grami_source_path+source_filename, Grami_des_path+des_profile_filename, Grami_des_path+des_relationships_filename))

In [ ]:
# Grami_source_path = 'kFoldData/CL_10K_1d8_L5/'
# Grami_des_path = 'kFoldData/CL_10K_1d8_L5/'
# data_convert_Grami('CL_10K_1d8_L5.lg', Grami_des_path)

In [ ]:
# MODIFY_INDEX = 0
# g = MyGraph()
# g.loadData('kFoldData/web_spam_detection/web_spam_detection_profiles.txt', 'kFoldData/web_spam_detection/web_spam_detection_relationships.txt')
# print(len(g.nodes), len(g.edges))

In [ ]:
# start = time.time()
# fc = FPMinerOpt8(g, 4835)
# print('Runtime: ', time.time()-start)
# fc.display()
# 250064 6314

In [ ]:
# fc.display()

In [ ]:
# rule = ruleGen(fc, 0.5)
# rule

In [ ]:
# source_path = "Data_demo/mico/"
# des_path = "GraMi/Datasets/"

# Grami_source_path = 'kFoldData/tweeter/'
# Grami_des_path = 'kFoldData/tweeter/'

# Convert data from mydataFormat to GramiFormat
def data_convert_to_Grami(profiles_filename, relationships_filename):
  pf_ = source_path + profiles_filename
  rf_ = source_path + relationships_filename
  des_filename = des_path + profiles_filename.split('.')[0] + '.lg'
  with open(pf_, 'r') as pf:
    with open(rf_, 'r') as rf:
      with open(des_filename, 'w') as f:
        vertexs_set = set()
        edges_set = set()

        f.write('t\t#\t1\n')
        profiles = pf.readlines()
        # print(profiles)
        for profile in profiles:
          data = profile.strip('\n').split('\t')
          # f.write('v {} {}\n'.format(data[0], data[1]))
          vertexs_set.add((int(data[0]), int(data[1])))
        vertexs_set = sorted(vertexs_set)
        for vertex in vertexs_set:
          v, l = vertex
          f.write('v {} {}\n'.format(v, l))
        print('Profiles! Done!')
        relationships = rf.readlines()
        # print(relationships)
        for relationship in relationships:
          data = relationship.strip('\n').split('\t')
          # f.write('e {} {} -1\n'.format(data[0], data[1]))
          edges_set.add((int(data[0]), int(data[1])))
        edges_set = sorted(edges_set)
        for edge in edges_set:
          v1, v2 = edge
          f.write('e {} {} -1\n'.format(v1, v2))
        print('Relationships! Done!')
  print('Done!')

In [ ]:
# source_path = 'kFoldData/CL_10K_1d8_L5/'
# des_path = 'kFoldData/CL_10K_1d8_L5/'
# data_convert_to_Grami('CL_10K_1d8_L5_profiles.txt', 'CL_10K_1d8_L5_relationships.txt')

# Measure Acc - OK

In [ ]:
def get_top_pair_node(G, measure_matrix, n):
  set_matrix = set()
  for i in range(len(measure_matrix)):
    for j in range(len(measure_matrix)):
      if measure_matrix[i][j] != 0.0 and measure_matrix[i][j]  != 1.0:
        set_matrix.add((measure_matrix[i][j], i, j))
        # set_matrix.add((i, j))
        
    result = set_matrix.copy()
    for edge in G.edges:
      v1, v2, l = edge
      for ed in set_matrix:
        w_, v1_, v2_ = ed
        if v1==v1_ and v2==v2_:
          result.remove(ed)
  result = sorted(result, reverse=True)
  return set(list(result)[:n])

def Jaccard_measure(node1, node2):
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  return len(set(node1.to_nodes).intersection(set(node2.to_nodes))) / len(list(set(node1.to_nodes) | set(node2.to_nodes)))

def cal_jaccard_measure_matrix(G):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Jaccard_measure(node1, node2)
  return result_matrix

def Jaccard_measure_v1(node1, node2):
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  return len(set(node1.to_nodes.union(node1.from_nodes)).intersection(set(node2.to_nodes.union(node2.from_nodes)))) / len(list(set(node1.to_nodes.union(node1.from_nodes)) | set(node2.to_nodes.union(node2.from_nodes))))

def cal_jaccard_measure_matrix_v1(G):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Jaccard_measure_v1(node1, node2)
  return result_matrix

def Simrank_measure(G, node1, node2, C):
  if node1.id == node2.id:
    return 1
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  sum_sim = 0
  for node1_ in G.nodes[node1.id].to_nodes:
    for node2_ in G.nodes[node2.id].to_nodes:
      sum_sim += Simrank_measure(G, G.nodes[node1_], G.nodes[node2_], C)
  return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix(G, C):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Simrank_measure(G, node1, node2, C)
  return result_matrix



In [ ]:
def Simrank_measure(G, node1, node2, C, iter_, max_iter):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes:
        for node2_ in G.nodes[node2.id].to_nodes:
            if iter_ <= max_iter:        
                sum_sim += Simrank_measure(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix(G, C, max_iter):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  count = 0
  for node1 in G.nodes:
    for node2 in G.nodes:
      count += 1
      if count%1000==0:
            print(count)
      result_matrix[node1.id][node2.id] = Simrank_measure(G, node1, node2, C, 0, max_iter)
  return result_matrix

def Simrank_measure_v1(G, node1, node2, C, iter_, max_iter, set_edge_checked):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes.union(G.nodes[node1.id].from_nodes):
        for node2_ in G.nodes[node2.id].to_nodes.union(G.nodes[node2.id].from_nodes):
            if iter_ <= max_iter:
                if (node1_, node2_) not in set_edge_checked:
                    set_edge_checked.add((node1_, node2_))
                    sum_sim += Simrank_measure_v1(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter, set_edge_checked)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix_v1(G, C, max_iter):
  # print('Init result matrix.................')
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  # count = 0
  # print('start: ')
  for node1 in G.nodes:
    # count += 1
    # if count%1000==0:
    #   print(count)
    for node2 in G.nodes:
      # print('edge: ', count, node1.id, node2.id, '.......................')
      result_matrix[node1.id][node2.id] = Simrank_measure_v1(G, node1, node2, C, 0, max_iter, set())
  return result_matrix

# .......................
def simrank(G, r=0.8, max_iter=100):
    result_matrix = [[1.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
    sim_old = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]

    # recursively calculate simrank
    for iter_ctr in range(max_iter):
        print(' - iter: ', iter_ctr)
        if _is_converge(result_matrix, sim_old):
              break
        print('Copy old matrix')
        sim_old = copy.deepcopy(result_matrix)#result_matrix.copy()
        print('start cal new matrix')
        bar = progressbar.ProgressBar(maxval=len(G.nodes)*len(G.nodes), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        bar.start()
        count = 0
        for u in range(0, len(G.nodes)):
            for v in range(0, len(G.nodes)):
                count += 1
                bar.update(count)
                if u == v:
                    continue
                s_uv = 0.0
                for n_u in G.nodes[u].to_nodes+G.nodes[u].from_nodes:
                    for n_v in G.nodes[v].to_nodes+G.nodes[v].from_nodes:
                        s_uv += sim_old[n_u][n_v]
                result_matrix[u][v] = r * s_uv / ((len(G.nodes[u].to_nodes)+len(G.nodes[u].from_nodes))*(len(G.nodes[v].to_nodes)+len(G.nodes[v].from_nodes)))
        bar.finish()
    return result_matrix

def _is_converge(s1, s2, eps=1e-4):
    for i in range(0, len(s1)):
        for j in range(0, len(s1)):
            if abs(s1[i][j] - s2[i][j]) >= eps:
                return False
    return True

def simrank_v1(G, r=0.8, max_iter=100):

  g = networkx.Graph()
  listEdges = []
  for edge in G.edges:
    v1, v2, _ = edge
    listEdges.append((v1, v2))
  g.add_edges_from(listEdges)

  # init. vars
  sim_old = defaultdict(list)
  sim = defaultdict(list)
  for n in g.nodes():
    sim[n] = defaultdict(int)
    sim[n][n] = 1
    sim_old[n] = defaultdict(int)
    sim_old[n][n] = 0

  # recursively calculate simrank
  for iter_ctr in range(max_iter):
    print('Iter ', iter_ctr)
    if _is_converge_v1(sim, sim_old):
      break
    sim_old = copy.deepcopy(sim)
    for u in g.nodes():
      for v in g.nodes():
        if u == v:
          continue
        s_uv = 0.0
        for n_u in g.neighbors(u):
          for n_v in g.neighbors(v):
            s_uv += sim_old[n_u][n_v]
        sim[u][v] = (r * s_uv / (len(list(g.neighbors(u))) * len(list(g.neighbors(v)))))

  result = [[0.0 for x in range(len(sim))] for y in range(len(sim))]
  for i in sim.keys():
    for j in sim[i].keys():
      result[i][j] = sim[i][j]
    
  return result

def _is_converge_v1(s1, s2, eps=1e-4):
  for i in s1.keys():
    for j in s1[i].keys():
      if abs(s1[i][j] - s2[i][j]) >= eps:
        return False
  return True

In [ ]:
def countEdgeGenInPredictedEdges(edgesGen, edgesPredicted):
    # edgesGen: gen from rule. {(v1, v2, vlv),....}
    # edgesPredicted: SimRank/Jaccard top-L predicted [(similarity, v1, v2),...]
    set_predicted = set()
    for edge in edgesPredicted:
        _, v1, v2 = edge
        set_predicted.add((v1, v2))
        
    result = 0
    for edge in edgesGen:
        v1, v2, vlv = edge
        if (v1, v2) in set_predicted:
            result += 1
    return result

In [ ]:
def get_predict_all_pair_node(G, measure_matrix):
    print('converting measure matrix to set......')
    set_matrix = set()
    for i in range(len(measure_matrix)):
        for j in range(len(measure_matrix)):
            if measure_matrix[i][j] != 0.0 and measure_matrix[i][j]  != 1.0:
                set_matrix.add((measure_matrix[i][j], i, j))
    
    # Init set of Edges for graph G
    print('Initing set of Edges for graph G..........')
    edgeG_set = set()
    for edge in G.edges:
        v1, v2, _ = edge
        edgeG_set.add((v1, v2))
 
    # remove edge that is in G.edges
    print('removing edge that is in G.edges.....')
    result = set_matrix.copy()
    for edge in set_matrix:
        v_, v1, v2 = edge
        if (v1, v2) in edgeG_set:
            result.remove(edge)
    # result = sorted(result, reverse=True)
    return set(list(result))

In [ ]:
def testPipeline(dataName, supp, cnf, minTopL, maxTopL, stepTopL, KFold, NTopRules, directed_=True):
    import progressbar
    result = {'jaccard': {}, 'simrank': {}}
    for i in range(0, KFold):
        print('----------------- Fold ', i, '--------------------')
        test = MyGraph()
        train = MyGraph()
        test.loadData(dataName+'_profiles_'+str(i)+'_test.txt', dataName+'_relationships_'+str(i)+'_test.txt')
        train.loadData(dataName+'_profiles_'+str(i)+'_train.txt', dataName+'_relationships_'+str(i)+'_train.txt')
        
        print('Minning FP.......')
        gc = train.FPMiner(supp)
        print('Gen Rules........')
        rules, confs = gc.ruleGen(cnf)

        if len(rules)==0:
            print('Fold: ', i, ' - Not rule!')
        # top_rule_idx = getNMaxrule(confs, NTopRules)
    
        print('Gen new Edges............')
        bar = progressbar.ProgressBar(maxval=len(rules), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        ttt = set()
        per = 0
        bar.start()
        for rule in rules:
            ttt = ttt.union(test.genNewEdges(rule))
            bar.update(per+1)
            per += 1
        bar.finish()
        
        print('Cal simrank_matrix........')
        simrank_matrix = simrank_v1(test, 0.8, 10)
        predict_simrank = get_predict_all_pair_node(test, simrank_matrix)
        predict_simrank = sorted(predict_simrank)

        print('Cal jaccard_matrix........')
        jaccard_matrix = cal_jaccard_measure_matrix_v1(test)
        predict_jaccard = get_predict_all_pair_node(test, jaccard_matrix)
        predict_jaccard = sorted(predict_jaccard)
        
        topL = minTopL
        while topL <= maxTopL:
            if topL not in result['jaccard']:
                result['jaccard'][topL] = {}
                result['simrank'][topL] = {}
            print('TopL = ', topL)
            Lrs = countEdgeGenInPredictedEdges(ttt, predict_simrank[-topL:])
            Lrj = countEdgeGenInPredictedEdges(ttt, predict_jaccard[-topL:])
            print('Jaccard: ', Lrj/topL, ' SimRank: ', Lrs/topL)
            print('Jaccard: ', Lrj/topL)
            result['jaccard'][topL][i] = Lrj/topL
            result['simrank'][topL][i] = Lrs/topL
            topL += stepTopL
    
    topL = minTopL
    while topL <= maxTopL:
        sum_sim = 0.0
        sum_jac = 0.0
        for i in range(0, KFold):
            sum_jac += result['jaccard'][topL][i]
            sum_sim += result['simrank'][topL][i]
        result['jaccard'][topL]['Mean'] = sum_jac/KFold
        result['simrank'][topL]['Mean'] = sum_sim/KFold
        topL += stepTopL
    
    with open(dataName+'_result_acc_top_'+str(NTopRules)+'rule_v5.json', 'w') as f:
        json.dump(result, f)
    
    return result

In [ ]:
# device_k_fold_v1('kFoldData/cora/cora_profiles.txt', 'kFoldData/cora/cora_relationships.txt', 4)

In [ ]:
# ! pwd

In [ ]:
# # # testPipeline('kFoldData/citeseer/citeseer', 100, 0.9, 100, 500, 50, 5, 5)
# data_pool = ['kFoldData/CL_10K_1d8_L5/CL_10K_1d8_L5', 'kFoldData/PLC_40_30_L5/PLC_40_30_L5', 'kFoldData/citeseer/citeseer', 'kFoldData/cora/cora', 'kFoldData/mico/mico', 'kFoldData/test1/test1', 'kFoldData/test2/test2'] 
# # sup_pool = [100, 150, 10500, *256*, 80]
# sup_pool = [350, 1715, 90, 190, 10500, 258, 80] # cora_new: not train, test1_new: not train, test2_new: not train
# conf_pool = [0.5, 0.9, 0.9, 0.6, 0.6, 0.9, 0.9]
# minTopL = [50, 50, 50, 50, 50, 50, 50]
# maxTopL = [500, 500, 500, 500, 500, 500, 500]
# stepTopL = [50, 50, 50, 50, 50, 50, 50]
# kFold = [4, 40, 40, 4, 5, 5, 5]
# nTopRule = [5, 5, 5, 5, 5, 5, 5]

# # [0, 1]: max_iter = 100
# # [2, 3, 4]: max_iter = 50

# for i in [3]:
#   print('#######################', data_pool[i], '##########################')
#   select = i
#   if i in [2, 3, 4]:
#     # print('---------------------------V3------------------------------------')
#     # testPipeline_v3(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # print('---------------------------V4------------------------------------')
#     # testPipeline_v4(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     print('---------------------------V5------------------------------------')
#     testPipeline_v5(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#   else:
#     # print('---------------------------V0------------------------------------')
#     # testPipeline(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # print('---------------------------V1------------------------------------')
#     # testPipeline_v1(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # print('---------------------------V2------------------------------------')
#     # testPipeline_v2(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # print('---------------------------V3------------------------------------')
#     # testPipeline_v3(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # print('---------------------------V4------------------------------------')
#     # testPipeline_v4(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     print('---------------------------V5------------------------------------')
#     testPipeline_v5(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])
#     # 28044 5429
#     # 34588 6110

# TEST GRAMI

In [ ]:
# % cd GraMi
# ! chmod +x grami

In [ ]:
# tweeter/retweet: [2900, 2600, 2500, 1800, 1850, 1700]
# ! ./grami -f retweet_profiles.lg -s 1700 -t 1 -p 1
# facebook/facebook: [3100, 3090, 3089, 3084, *3081, *3080]
# ! ./grami -f facebook_profiles.lg -s 3081 -t 1 -p 1
# webkb_wisc/webkb_wisc - nodes/edges/labels=265/530/5: []
# ! ./grami -f webkb_wisc.lg -s 200 -t 1 -p 1
# web_spam_detection/web_spam_detection - nodes/edges/labels=9072/514700/3: []
# ! ./grami -f web_spam_detection.lg -s 50000 -t 1 -p 1
# CL_10K_1d8_L5/CL_10K_1d8_L5 - nodes/edges/labels=10000/44896/5: [700, 600, 500, 450, 400, ] ook - s:400, conf:0.5->0.6
# ! ./grami -f CL_10K_1d8_L5.lg -s 400 -t 1 -p 1
# CL_10K_1d8_L5/CL_10K_1d8_L5 - nodes/edges/labels=10000/44896/5: [700, 600, 500, 450, 400, ] 
# ! ./grami -f CL_10K_1d8_L5.lg -s 400 -t 1 -p 1

In [ ]:
# cat Output.txt

In [ ]:
# % cd ..

In [ ]:
# ! ./grami -f example.lg -s 2 -t 1 -p 1